In [1]:
import pandas as pd

In [4]:
model_input = pd.read_csv("C:/Users/ruchi/OneDrive/Ruchira-MAIN/Work/Stats_ML/model_input_data_v2.csv")

In [5]:
model_input.head()

,Unnamed: 0,adjusted_date,ticker,transcript,abret_1d,abret_5d,abvol_20d,abret_callday,r1d_direction,r5d_direction
0,2,2019-10-24,LRCX,Prepared Remarks:\nOperator\nGood day and welc...,0.012659,0.010627,0.032240,0.137401,1,1
1,11,2021-08-26,SJM,"Prepared Remarks:\nOperator\nGood morning, and...",-0.011765,-0.031104,0.013258,-0.020496,0,0
2,19,2019-11-07,DXCM,Prepared Remarks:\nOperator\nWelcome to the De...,0.014894,0.037208,0.064064,0.268035,1,1
3,24,2021-07-01,WBA,Prepared Remarks:\nOperator\nLadies and gentle...,-0.018729,-0.038514,0.020677,-0.079667,0,0
4,25,2021-05-04,ZBRA,"Prepared Remarks:\nOperator\nGood day, and wel...",0.024761,0.035372,0.013530,-0.011428,1,1


In [7]:
import numpy as np
df = model_input.copy()

df = df.sort_values("adjusted_date").reset_index(drop=True)
dates = np.array(sorted(df["adjusted_date"].unique()))
n_dates = len(dates)

train_end = int(0.75 * n_dates)
val_end   = int(0.87 * n_dates)  # 65% + 10%

train_dates = dates[:train_end]
val_dates   = dates[train_end:val_end]
test_dates  = dates[val_end:]

train_df = df[df["adjusted_date"].isin(train_dates)].reset_index(drop=True)
val_df   = df[df["adjusted_date"].isin(val_dates)].reset_index(drop=True)
test_df  = df[df["adjusted_date"].isin(test_dates)].reset_index(drop=True)

print("Unique dates:", n_dates)
print("Rows:", len(train_df), len(val_df), len(test_df))
print("Train range:", train_dates[0], "→", train_dates[-1])
print("Val range:  ", val_dates[0], "→", val_dates[-1])
print("Test range: ", test_dates[0], "→", test_dates[-1])


Unique dates: 511
Rows: 2823 319 402
Train range: 2019-04-17 → 2022-02-10
Val range:   2022-02-11 → 2022-07-14
Test range:  2022-07-15 → 2022-12-01


In [8]:
y_train_1d = train_df["r1d_direction"]
y_val_1d   = val_df["r1d_direction"]
y_test_1d  = test_df["r1d_direction"]

y_train_5d = train_df["r5d_direction"]
y_val_5d   = val_df["r5d_direction"]
y_test_5d  = test_df["r5d_direction"]

features_base_train = train_df[["abvol_20d", "abret_callday"]]
features_base_val   = val_df[["abvol_20d", "abret_callday"]]
features_base_test  = test_df[["abvol_20d", "abret_callday"]]

# Baseline Model-0 (Random)

In [11]:
import random as rnd
rng = np.random.default_rng(seed=42)

In [10]:
p_5d_train=y_train_5d.sum()/len(y_train_5d)
p_5d_train

np.float64(0.49273822174991144)

In [30]:
p_pred_random_5d= rng.uniform(size=len(y_test_5d)) #generates probabilities randomly
y_pred_random_5d = (p_pred_random_5d >= 1-p_5d_train).astype(int) #converts probabilities to class labels based on 1-p_5d_train threshold

In [31]:
from sklearn.metrics import accuracy_score,roc_auc_score,confusion_matrix
accuracy_random_5d = accuracy_score(y_test_5d, y_pred_random_5d)
auc_random_5d=roc_auc_score(y_test_5d, p_pred_random_5d)

print("Random Model 5d - Accuracy:", accuracy_random_5d)
print("Random Model 5d - AUC:", auc_random_5d)

Random Model 5d - Accuracy: 0.4975124378109453
Random Model 5d - AUC: 0.49710123008671103


# Baseline-1 Finance Data only